__How to train a language model__	Notebook to Highlight all the steps to effectively train Transformer model on custom data
https://github.com/huggingface/transformers/tree/master/notebooks

https://github.com/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb

__Language Modeling__
https://github.com/huggingface/transformers/tree/master/examples/language-modeling
https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py

In [7]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

from pathlib import Path
import torch

from torch.utils.data import Dataset, DataLoader
from tokenizers import CharBPETokenizer, Tokenizer, ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers.normalizers import BertNormalizer
# from tokenizers import SentencePieceBPETokenizer

import random
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast , AutoTokenizer,RobertaTokenizerFast, RobertaTokenizer
from filelock import FileLock
import logging
import time
import tqdm
import pickle
from multiprocessing import Pool
from functools import reduce

In [2]:
array = ["cat", "dog", "mat", "pat", "rat", "bat" , "bolo"]
def testadder(arrItem):
    return arrItem+"123"

with Pool(processes=3) as p:
    examples = p.map(testadder, array)
print(examples)

['cat123', 'dog123', 'mat123', 'pat123', 'rat123', 'bat123', 'bolo123']


In [3]:
# device = torch.device("cuda:1")
# device

In [4]:
!nvidia-smi

Sun Jun 28 20:09:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:21:01.0 Off |                    0 |
| N/A   28C    P0    54W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:21:02.0 Off |                    0 |
| N/A   

In [5]:
# Check that PyTorch sees it
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
8


In [9]:
DATA_PATH = Path("..")

# DATA_RAW_PATH = DATA_PATH/"raw"
DATA_RAW_EXTRACTED_PATH = DATA_PATH/"raw_data_extraction"

# Output is in bytes - helper from Pathlib Path https://stackoverflow.com/questions/2104080/how-can-i-check-file-size-in-python
def getStat(prev_value, cur_value):
    if isinstance(prev_value, int):
        return prev_value + cur_value.stat().st_size
    return prev_value.stat().st_size + cur_value.stat().st_size

# 1. The data from thwiki
THWIKI_FOLDER = Path("thwiki-20200601-extracted")
WIKI_FILES = list((DATA_RAW_EXTRACTED_PATH/THWIKI_FOLDER).glob("*.txt"))
list(map(print , WIKI_FILES[:5]))
print(f"thwiki-20200601-extracted Amounts to a total of {reduce(getStat, WIKI_FILES)/1e6:.2f} MB")

# 2. The classification data from jung and ninja
CLASSIFICATION_JUNG_NINJA_FOLDER = Path("classification_dataset")
CLASSIFICATION_FILES = list((DATA_RAW_EXTRACTED_PATH/CLASSIFICATION_JUNG_NINJA_FOLDER).glob("*.txt"))
list(map(print , CLASSIFICATION_FILES[:5]))
print(f"classification_dataset Amounts to a total of {reduce(getStat, CLASSIFICATION_FILES)/1e6:.2f} MB")

# 3. The Data from p'Moo Crawlers
ANOTHER_WEBSITE_MOO_FOLDER = Path("another_website")
ANOTHER_WEBSITE_FILES = list((DATA_RAW_EXTRACTED_PATH/ANOTHER_WEBSITE_MOO_FOLDER).glob("*.txt"))
list(map(print , ANOTHER_WEBSITE_FILES[:5]))
print(f"another_website Amounts to a total of {reduce(getStat, ANOTHER_WEBSITE_FILES)/1e6:.2f} MB")

# 4. Senior Project Files
SENIOR_PROJ_FOLDER = Path("data_lm")
SENIOR_PROJ_FILES = list((DATA_RAW_EXTRACTED_PATH/SENIOR_PROJ_FOLDER).glob("*.txt"))
list(map(print , SENIOR_PROJ_FILES[:5]))
print(f"Senior Project Amounts to a total of {reduce(getStat, SENIOR_PROJ_FILES)/1e6:.2f} MB")

# 5. Guru Crawler Files
GURU_CRAWLER_FOLDER = Path("social_listening")
GURU_CRAWLER_FILES = list((DATA_RAW_EXTRACTED_PATH/GURU_CRAWLER_FOLDER).glob("*.txt"))
list(map(print , GURU_CRAWLER_FILES[:5]))
print(f"GuruCrawler Amounts to a total of {reduce(getStat, GURU_CRAWLER_FILES)/1e6:.2f} MB")

ALL_FILES = WIKI_FILES + CLASSIFICATION_FILES + ANOTHER_WEBSITE_FILES + SENIOR_PROJ_FILES + GURU_CRAWLER_FILES
print(f"\nI have a total of {len(ALL_FILES)} files!")





print(f"Amounts to a total of {reduce(getStat, ALL_FILES)/1e6:.2f} MB")

../raw_data_extraction/thwiki-20200601-extracted/WikiAD_3.txt
../raw_data_extraction/thwiki-20200601-extracted/WikiAE_3.txt
../raw_data_extraction/thwiki-20200601-extracted/WikiAF_0.txt
../raw_data_extraction/thwiki-20200601-extracted/WikiAF_2.txt
../raw_data_extraction/thwiki-20200601-extracted/WikiAD_1.txt
thwiki-20200601-extracted Amounts to a total of 566.79 MB
../raw_data_extraction/classification_dataset/dailynews_0.txt
../raw_data_extraction/classification_dataset/pptv36_0.txt
../raw_data_extraction/classification_dataset/prbangkok_0.txt
../raw_data_extraction/classification_dataset/siamrath_0.txt
../raw_data_extraction/classification_dataset/springnews_0.txt
classification_dataset Amounts to a total of 50.79 MB
../raw_data_extraction/another_website/khaosod_16.txt
../raw_data_extraction/another_website/pantip_470.txt
../raw_data_extraction/another_website/pantip_415.txt
../raw_data_extraction/another_website/naewna_2.txt
../raw_data_extraction/another_website/brighttv_5.txt
ano

# Making Electra Model

However, currently pretraining Electra is still inside PR stage  
- Issue : When will ELECTRA pretraining from scratch will be available? #3878 https://github.com/huggingface/transformers/issues/3878  
- Issue : BERT and other models pretraining from scratch example #4425 https://github.com/huggingface/transformers/issues/4425
- PR : Electra training from scratch #4656 https://github.com/huggingface/transformers/pull/4656

Combined model
- Combines ElectraForMaskedLM and ElectraForPreTraining with embedding sharing + custom masking/replaced token detection

Commit before Merge: https://github.com/huggingface/transformers/pull/4656/commits/30b2dbbba6918ac6540b6a1758b7ee19f0ac969c#diff-8a95e2bfb7da25648b5d12ffa69fd7a3

In [10]:
# from transformers import ElectraModel, ElectraConfig

# # Initializing a ELECTRA electra-base-uncased style configuration
# configuration = ElectraConfig()
# configuration.vocab_size = 20000

# # Initializing a model from the electra-base-uncased style configuration
# model = ElectraModel(configuration)

# # Accessing the model configuration
# configuration = model.config
# configuration

In [11]:
# model

In [12]:
# model.num_parameters()
# # => 12 million parameters

In [13]:
# model.get_input_embeddings()

# Trying out Roberta per Notebook 

From __HuggingFace Notebooks__ https://huggingface.co/transformers/notebooks.html: 

How to train a language model	Highlight all the steps to effectively train Transformer model on custom data
- Colab (ipynb) version : https://github.com/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb
- MD version: https://github.com/huggingface/blog/blob/master/how-to-train.md

Pretrain Longformer	How to build a "long" version of existing pretrained models	Iz Beltagy  
https://github.com/allenai/longformer/blob/master/scripts/convert_model_to_long.ipynb

In [14]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

configuration = RobertaConfig(
    vocab_size=30522,
    max_position_embeddings=512, # 512 + 2 more special tokens
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=1,
)
# configuration.vocab_size = 20000

model = RobertaForMaskedLM(config=configuration)

# Accessing the model configuration
model.config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 30522
}

In [15]:
model.num_parameters()
# => 102 million parameters

110104122

In [16]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

# Initializing Tokenizer

In [17]:
# from tokenizers import Tokenizer
# tokenizer = Tokenizer.from_file("./thwiki-sentencepiecebpe.tokenizer.json")
# encoded =  tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces")
# print(encoded.ids)
# print(encoded.tokens)

In [18]:
# tokenizer.enable_truncation(max_length=128)

In [19]:
# encoded =  tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง SpacesWhat is great is that our tokenizer is optimized for Esperanto. Compared to a generic tokenizer trained for English, more native words are represented by a single, unsplit token. Diacritics, i.e. accented characters used in Esperanto – ĉ, ĝ, ĥ, ĵ, ŝ, and ŭ – are encoded natively. We also represent sequences in a more efficient manner. Here on this corpus, the average length of encoded sequences is ~30% smaller as when using the pretrained GPT-2 tokenizer.")
# print("This will not be over 128: ", len(encoded.ids), encoded.tokens)
# print(encoded.overflowing[0].tokens)

wrap tokenizers inside a PreTrainedTokenizerFast from transformers 

https://github.com/huggingface/tokenizers/issues/259

In [20]:
# class SentencePieceBPETokenizerFast(PreTrainedTokenizerFast):
#     def __init__(
#         self,
#         vocab_file,
#         merges_file,
#         bos_token="<s>",
#         eos_token="</s>",
#         sep_token="</s>",
#         cls_token="<s>",
#         unk_token="<unk>",
#         pad_token="<pad>",
#         mask_token="<mask>",
#         **kwargs
#     ):
#         super().__init__(
#             SentencePieceBPETokenizer(
#                 vocab_file=vocab_file,
#                 merges_file=merges_file,
#             ),
#              bos_token=bos_token,
#             eos_token=eos_token,
#             unk_token=unk_token,
#             sep_token=sep_token,
#             cls_token=cls_token,
#             pad_token=pad_token,
#             mask_token=mask_token,
#             **kwargs,
#         )

In [21]:
# import json

# # with open("./thwiki-sentencepiecebpe.tokenizer.json", 'r' ) as json_data:
# with open("./thwiki-charbpe-30522.tokenizer.json", 'r' ) as json_data:
#      data = json.load(json_data)
# vocab = data['model']['vocab']
# merges = data['model']['merges']


# with open('vocab.json', 'w', encoding='utf-8') as json_file:
#     json.dump(vocab, json_file, ensure_ascii=False)
# with open('merges.txt', 'w', encoding='utf-8') as f:
#     for merge_string in merges:
#         f.write(f'{merge_string}\n')

In [22]:
# pretrain_tokenizer = SentencePieceBPETokenizerFast(vocab_file='vocab.json',merges_file ='merges.txt' )

In [23]:
# from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
# from tokenizers import Tokenizer
# from tokenizers.implementations import BaseTokenizer

# tokenizer = Tokenizer.from_file("./thwiki-sentencepiecebpe.tokenizer.json")
# base_tokenizer = BaseTokenizer(tokenizer) # Wrapper!! to PretrainTokenizerFast Tokenizer should be an instance of a Tokenizer provided by HuggingFace tokenizers library.
# base_tokenizer = SentencePieceBPETokenizer()
# pretrain_tokenizer = PreTrainedTokenizerFast(tokenizer=base_tokenizer)
# pretrain_tokenizer

In [24]:
# from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
# from tokenizers import Tokenizer, CharBPETokenizer
# from tokenizers.implementations import BaseTokenizer

# # tokenizer = Tokenizer.from_file("./thwiki-charbpe-30522.tokenizer.json")
# # base_tokenizer = BaseTokenizer(tokenizer) # Wrapper!! to PretrainTokenizerFast Tokenizer should be an instance of a Tokenizer provided by HuggingFace tokenizers library.
# base_tokenizer = CharBPETokenizer(vocab_file='vocab.json',merges_file ='merges.txt')
# pretrain_tokenizer = PreTrainedTokenizerFast(tokenizer=base_tokenizer)
# pretrain_tokenizer

# CANNOT USE FAST BECAUSE multiprocessing Pool doesn't support Rust!!
Using %%time, there is almost x7-x10 difference in tokenization but we cannot not allow multiprocessing

In [25]:
# from transformers import RobertaTokenizerFast

# tokenizer = RobertaTokenizerFast.from_pretrained("./thwiki-seniorproj-bytebpe-30522", max_len=512)

In [71]:
# from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("./thwiki-seniorproj-bytebpe-30522", max_len=512)
tokenizer.__class__.__name__

'RobertaTokenizer'

In [72]:
print(tokenizer.cls_token_id)
print(tokenizer.num_special_tokens_to_add())
tokenized_text = tokenizer.tokenize("ฉันเคยเกือบพลาดสิ่งที่ดีที่สุดในชีวิต หากในวันที่ฉันล้มอยู่ ไม่มีหนึ่งใจของเธอ ฝันคงจบ หลายสิ่งที่ดีคงหมดทางได้เจอ หนึ่งกำลังใจที่ยิ่งใหญ่ ไม่ลืมได้เลย... ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ  ก่อนอื่นผมต้องบอกก่อนเลยว่าคนเราจะเลือกกินอาหารแบบไหนชอบแบบไหนเป็นเรื่องของความชอบส่วนตัวนะครับทุกคนมีสิทธิในการเลือกของที่ชอบและไม่ชอบอยู่แล้ว แต่ผมรู้สึกว่าตอนนี้ผมกำลังประสบปัญหาที่ดูเหมือนจะเล็กแต่กลายเป็นว่ามันค่อนข้างใหญ่ ผมคบกับแฟนมา6ปีแล้วครับ ผมเป็นคนชอบกินอาหารญี่ปุ่นและปลาดิบแต่แฟนผมไม่กินปลาดิบเลย ผมอยากกินบุฟเฟ่เนื้อแต่แฟนผมก็ไม่กินเนื้อ เราเลยไม่ได้เข้าทานร้านบุฟเฟ่เนื้อและบุฟเฟ่อาหารญี่ปุ่นกันเพราะรู้สึกลัวแฟนผมทานไม่คุ้ม และเรื่องใหญ่เลยคือผมเป็นคนชอบทานอาหารรสจัดและรสเผ็ดมาก แต่แฟนผมทานเผ็ดไม่ได้เลยเวลาเราไปกินส้มตำกันก็จะสั่ง ส้มตำไม่ใส่พริก ต้มแซ่บไม่ใส่พริก ลาบไม่ใส่พริก ร้านกับข้าวอื่นๆก็เช่นกันแฟนผมจะไม่ชอบกินผักไม่ค่อยสั่งกับข้าวที่เป็นผักแล้วผมชอบผักบุ้งทอดกรอบ เห็ดหอมสดทอดมาก แต่ก็ไม่ได้สั่งเพราะว่าเธอไม่กินถึงเค้าจะบอกให้สั่งเลยๆก็เถอะแต่ผมก็ยังเกรงใจเธออยู่ดีอ่ะครับ ผมรู้สึกกินอาหารไม่มีความสุขเลยชีวิตผมขาดรสเผ็ดไปเหมือนจะขาดใจเหมือนมันทำให้ขาดความสุขไปอย่างนึงเลยอ่ะครับ ยิ่งถ้าเราแต่งงานกันแล้วผมก็อาจจะต้องมีปัญหาเรื่องนี้มากขึ้น พอผมเห็นคู่ที่ชอบทานอาหารเหมือนๆกันเห็นเค้ากินอาหารกันอย่างมีความสุขแล้วผมรู้สึกอิจฉามากๆเลย มีใครเคยมีปัญหาแบบผมมั้ยครับแล้วจะแก้ปัญหานี้ยังไงดีครับ")
print(f"{tokenized_text} with length {len(tokenized_text)}")
tokenized_text = tokenizer.convert_tokens_to_ids(tokenized_text)
print(f"{tokenized_text} with length {len(tokenized_text)}")
print(tokenizer.build_inputs_with_special_tokens(tokenized_text))

0
2
['à¸ī', 'à¸±', 'à¸Ļà¹Ģà¸Ħà¸¢', 'à¹Ģà¸ģ', 'à¸·', 'à¸Ńà¸ļ', 'à¸ŀà¸¥à¸²à¸Ķ', 'à¸ª', 'à¸´à¹Ī', 'à¸ĩà¸Ĺ', 'à¸µà¹Ī', 'à¸Ķ', 'à¸µ', 'à¸Ĺ', 'à¸µà¹Ī', 'à¸ª', 'à¸¸', 'à¸Ķà¹ĥà¸Ļà¸Ĭ', 'à¸µ', 'à¸§', 'à¸´', 'à¸ķ', 'Ġà¸«à¸²à¸ģ', 'à¹ĥà¸Ļà¸§', 'à¸±', 'à¸Ļà¸Ĺ', 'à¸µà¹Ī', 'à¸ī', 'à¸±', 'à¸Ļà¸¥', 'à¹ī', 'à¸¡à¸Ńà¸¢', 'à¸¹à¹Ī', 'Ġà¹Ħà¸¡', 'à¹Ī', 'à¸¡', 'à¸µ', 'à¸«à¸Ļ', 'à¸¶à¹Ī', 'à¸ĩà¹ĥà¸Ī', 'à¸Ĥà¸Ńà¸ĩà¹Ģà¸ĺà¸Ń', 'Ġà¸Ŀ', 'à¸±', 'à¸Ļà¸Ħ', 'à¸ĩà¸Īà¸ļ', 'Ġà¸«à¸¥à¸²à¸¢', 'à¸ª', 'à¸´à¹Ī', 'à¸ĩà¸Ĺ', 'à¸µà¹Ī', 'à¸Ķ', 'à¸µ', 'à¸Ħ', 'à¸ĩà¸«à¸¡à¸Ķ', 'à¸Ĺà¸²à¸ĩ', 'à¹Ħà¸Ķ', 'à¹ī', 'à¹Ģà¸Īà¸Ń', 'Ġà¸«à¸Ļ', 'à¸¶à¹Ī', 'à¸ĩà¸ģ', 'à', '¸', '³', 'à¸¥', 'à¸±', 'à¸ĩà¹ĥà¸Īà¸Ĺ', 'à¸µà¹Ī', 'à¸¢', 'à¸´à¹Ī', 'à¸ĩà¹ĥà¸«à¸į', 'à¹Ī', 'Ġà¹Ħà¸¡', 'à¹Ī', 'à¸¥', 'à¸·', 'à¸¡à¹Ħà¸Ķ', 'à¹ī', 'à¹Ģà¸¥à¸¢', '...', 'Ġà¹ĥà¸Ħà¸£à¹Ģà¸Ħà¸¢à¸¡', 'à¸µ', 'à¹ģà¸Łà¸Ļà¸Ĺ', 'à¸µà¹Ī', 'à¸ģ', 'à¸´', 'à¸Ļà¸Ńà¸²à¸«à¸²à¸£', 'à¹Ħà¸¡', 'à¹Ī', 'à¸ĸ', 'à¸¹', 'à¸ģà¸Ľ', 'à¸²à¸ģà¸ģ', 'à¸±', 'à¸Ļà¹ģà¸¥', 'à¹ī', 'à¸§à¸£', 'à¸¹à¹ī', 'à¸ª', 'à¸¶', 'à¸ģà¹Ģà¸ª', 'à¸µ', '

Constructing tokenizer wrapper based on [@theblackcat102 #259](https://github.com/huggingface/tokenizers/issues/259#issuecomment-625905930)

In [28]:
# class RobertaTokenizerFast(PreTrainedTokenizerFast):

#     def __init__(
#         self,
#         vocab_file,
#         merges_file,
#         bos_token="<s>",
#         eos_token="</s>",
#         sep_token="</s>",
#         cls_token="<s>",
#         unk_token="<unk>",
#         pad_token="<pad>",
#         mask_token="<mask>",
#         **kwargs
#     ):
#         super().__init__(
#             ByteLevelBPETokenizer(vocab_file=vocab_file,
#                                     merges_file=merges_file,
#                                   lowercase=True,                 # adds lowercase to normalizer
#                                   unicode_normalizer="nfkc",      # adds unicode normalizer nfkc to normalizer
#                                   add_prefix_space=True,          # add space to the first word
#                                  ),
#              bos_token=bos_token,
#             eos_token=eos_token,
#             unk_token=unk_token,
#             sep_token=sep_token,
#             cls_token=cls_token,
#             pad_token=pad_token,
#             mask_token=mask_token,
#             **kwargs,
#         )

In [29]:
# tokenizer3 = RobertaTokenizerFast(vocab_file= "./thwiki-seniorproj-bytebpe-30522/vocab.json",
#                                  merges_file="./thwiki-seniorproj-bytebpe-30522/merges.txt")
# print(tokenizer3.num_special_tokens_to_add())


In [30]:
# tokenizer2 = Tokenizer.from_file("./thwiki-seniorproj-bytebpe-30522.tokenizer.json")
# tokenizer2.num_special_tokens_to_add(is_pair=False)

In [31]:
# tokenizer2.normalize("asdpo qweasd")

# Building our dataset

Build it with `from torch.utils.data.dataset import Dataset` just like [TextDataset](https://github.com/huggingface/transformers/blob/448c467256332e4be8c122a159b482c1ef039b98/src/transformers/data/datasets/language_modeling.py) and [LineByLineTextDataset](https://github.com/huggingface/transformers/blob/448c467256332e4be8c122a159b482c1ef039b98/src/transformers/data/datasets/language_modeling.py#L78)

Note: Training with multiple files is currently not supported [issue/3445](https://github.com/huggingface/transformers/issues/3445)

padding documentation [link](https://github.com/huggingface/tokenizers/blob/master/bindings/python/tokenizers/implementations/base_tokenizer.py#L52)

Potential Improvements
- การทำให้ Dataset นั้น dynamically tokenize + dynamically open file : ตอนนี้เวลาทำ Dataset จาก torch.utils.data.dataset จะทำการ tokenize เลยตอนอยู่ใน constructor  , กำลังคิดว่าถ้าเกิดว่า Data ใหญ่มากๆ อาจจะไม่เหมาะสมกับการทำแบบนี้  เพราะว่า Ram จะต้องมีขนาดเท่าๆกับ data ที่เราใส่เข้าไป  ซึ่งเป็นไปได้ยากหาก Data มีขนาดใหญ่มากๆ   ผมได้ทำการ Search ดูแล้วก็พบว่าจาก Discussion Forum ของ Pytorch: https://discuss.pytorch.org/t/how-to-use-a-huge-line-corpus-text-with-dataset-dataloader/30872 
Option1: ใช้ pd.Dataframe ในการเปิด File แบบ small chunks of data https://discuss.pytorch.org/t/data-processing-as-a-batch-way/14154/4?u=ptrblck
Option2: ใช้ byte Offsets จากไฟล์ใหญ่ๆเพื่อที่จะ lookup .seek(): https://github.com/pytorch/text/issues/130#issuecomment-510412877
More Examples: https://github.com/pytorch/text/blob/master/torchtext/datasets/unsupervised_learning.py , https://github.com/pytorch/text/blob/a5880a3da7928dd7dd529507eec943a307204de7/examples/text_classification/iterable_train.py#L169-L214

In [54]:
logger = logging.getLogger(__name__)


class TextDatasetParallel(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """         
    def __init__(self, tokenizer: PreTrainedTokenizer, sample_path: [], block_size: int, overwrite_cache=False,
                num_processes=8, cached_directory = "/workdir/Code/bma_transformer_model/data/cached_data"):
        # assert os.path.isfile(file_path)
        # For Loop MultiFile
        self.examples = []
        self.sample_path = sample_path
#         print(f"THIS IS SAMPLE PATH {sample_path}")
        self.tokenizer = tokenizer
        
        # Set block size to be the blocksize-special tokens
        self.block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)
        
        self.overwrite_cache = overwrite_cache
        self.cached_directory = cached_directory
        if not os.path.exists(cached_directory):
            os.makedirs(cached_directory)
        
        # Multiprocess for getting examples
        with Pool(processes=num_processes) as p:
            self.examples = list(tqdm.tqdm(p.imap(self.load_data_tokenized, self.sample_path), total=len(self.sample_path)))
        
        

    def load_data_tokenized(self, file_path):
#         print(f"I AM DOING {file_path}")
        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            self.cached_directory, f"cached_lm_{tokenizer.__class__.__name__}_{str(self.block_size)}_{filename}",
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):
            if os.path.exists(cached_features_file) and not self.overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    temp_examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )
            else:
                temp_examples = []
                with open(file_path, encoding="utf-8") as f:
                    text = f.read()
                print("I finished reading ", file_path)
                tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[:10]))
                print("I finished tokenizing ", file_path)
                for i in range(0, len(tokenized_text) - self.block_size + 1, self.block_size):  # Truncate in block of block_size
                    temp_examples.append(
                        tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + self.block_size])
                    )
                    if i%20 == 0:
                        print("I finished special tok ", file_path)
                print("I finished every tokenizing ", file_path)
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should loook for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(temp_examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )
        return temp_examples
        
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)


Tokenizer from Pretrained copied from SimpleTransformers [link](https://github.com/ThilinaRajapakse/simpletransformers/blob/master/simpletransformers/language_modeling/language_modeling_model.py)

In [84]:
dataset = TextDatasetParallel(tokenizer, 
                              sample_path=list(map(str, GURU_CRAWLER_FILES)), 
                              block_size=512, 
                              cached_directory= "/workdir/cached_data",
                              overwrite_cache=False, # make sure this is false when you have cache!!
                              num_processes=3,
                             )







  0%|          | 0/6 [00:00<?, ?it/s]





 17%|█▋        | 1/6 [00:00<00:01,  2.52it/s]





 33%|███▎      | 2/6 [00:00<00:01,  2.97it/s]





 50%|█████     | 3/6 [00:00<00:00,  3.51it/s]





 67%|██████▋   | 4/6 [00:00<00:00,  3.86it/s]





100%|██████████| 6/6 [00:01<00:00,  5.14it/s]


In [73]:
# cached_directory= "/workdir/cached_data"
# def load_data_tokenized(file_path):
#     directory, filename = os.path.split(file_path)
#     cached_features_file = os.path.join(
#         cached_directory, f"cached_lm_something_{str(123)}_{filename}",
#     )
#     temp_examples = []
#     with open(file_path, encoding="utf-8") as f:
#         text = f.read()
#     print("I finished reading ", file_path)
#     tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[:10]))
#     print("I finished tokenizing ", file_path)
#     return f"{file_path}+123"
# # Multiprocess for getting examples
# with Pool(processes=2) as p:
#     examples = list(tqdm.tqdm(p.imap(load_data_tokenized, list(map(str, GURU_CRAWLER_FILES))), total=len(list(map(str, GURU_CRAWLER_FILES)))))
# print(examples)


In [82]:
# %%time
# print(text[:1000])
# print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[:1000])))

In [77]:
%%time
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[:100000]))
None

CPU times: user 900 ms, sys: 4 ms, total: 904 ms
Wall time: 903 ms


In [78]:
%%time
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[:1000000]))
None

CPU times: user 7.27 s, sys: 40 ms, total: 7.31 s
Wall time: 7.31 s


__For text[:3000000]__  

Rust implementation
>CPU times: user 6.38 s, sys: 328 ms, total: 6.7 s  
Wall time: 5.18 s

Python
>CPU times: user 15.5 s, sys: 72 ms, total: 15.6 s  
Wall time: 15.6 s

In [79]:
%%time
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[:3000000]))
None

CPU times: user 15.5 s, sys: 72 ms, total: 15.6 s
Wall time: 15.6 s


__For text[:8000000]__  

Rust implementation
>

Python
>CPU times: user 36.1 s, sys: 340 ms, total: 36.4 s  
Wall time: 36.4 s

In [80]:
%%time
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[:8000000]))
None

CPU times: user 36.1 s, sys: 340 ms, total: 36.4 s
Wall time: 36.4 s


In [43]:
len(text)

8055389

In [ ]:
dataset

In [27]:
# for i_batch, sample_batched in enumerate(dataloader):
#     print(i_batch, sample_batched)
#     oumodel()

In [28]:
# tokenizer = CharBPETokenizer(vocab_file='vocab.json',merges_file ='merges.txt' )
# no_accent_strip = BertNormalizer(strip_accents=False)
# tokenizer._tokenizer.normalizer = no_accent_strip
# tokenizer._tokenizer.post_processor = BertProcessing(
#     ("</s>", tokenizer.token_to_id("</s>")),
#     ("<s>", tokenizer.token_to_id("<s>")),
# )

# input_ids = torch.tensor(tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces").ids).unsqueeze(0)
# print(input_ids)
# outputs = model(input_ids, labels=input_ids)
# print(outputs)
# loss, prediction_scores = outputs[:2]
# print(loss, prediction_scores.shape)

In [29]:
# dataset.__getitem__(1).unsqueeze(0)

In [30]:
# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1

In [31]:
# %%time
# from transformers import TextDataset, LineByLineTextDataset

# # dataset = LineByLineTextDataset(
# #     tokenizer=pretrain_tokenizer,
# #     file_path="../data/text/AA/wiki_01",
# #     block_size=128,
# # )

# dataset = TextDataset(
#     tokenizer=pretrain_tokenizer,
#     file_path="../data/text/AA/wiki_01",
#     block_size=128,
# )


In [32]:
# one_doc = list(Path("../data/text/AA/").glob("wiki*"))[0].read_text(encoding="utf-8").splitlines()
# tokenizer = Tokenizer.from_file("./thwiki-sentencepiecebpe.tokenizer.json")
# tokenizer.encode_batch(one_doc[:8])

In [33]:
# one_doc = list(Path("../data/text/AA/").glob("wiki*"))[0].read_text(encoding="utf-8").splitlines()
# tokenizer = RobertaTokenizerFast(vocab_file='vocab.json',merges_file ='merges.txt', max_len=512)
# tokenizer.batch_encode_plus(one_doc[:8])

In [34]:
# print(tokenizer.encode_batch(one_doc[:8])[5].tokens)

In [35]:
# one_doc[:8]

In [36]:
from transformers import DataCollatorForLanguageModeling
# tokenizer = RobertaTokenizer(vocab_file='vocab.json',merges_file ='merges.txt', max_len=512)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Transfomers Trainer [link](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py#L133)

```python
class Trainer:
    """
    Trainer is a simple but feature-complete training and eval loop for PyTorch,
    optimized for Transformers.
    Args:
        prediction_loss_only:
            (Optional) in evaluation and prediction, only return the loss
    """
    def __init__(
        self,
        model: PreTrainedModel,
        args: TrainingArguments,
        data_collator: Optional[DataCollator] = None,
        train_dataset: Optional[Dataset] = None,
        eval_dataset: Optional[Dataset] = None,
        compute_metrics: Optional[Callable[[EvalPrediction], Dict]] = None,
        prediction_loss_only=False,
        tb_writer: Optional["SummaryWriter"] = None,
        optimizers: Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR] = None,
```

[TrainingArguments](https://github.com/huggingface/transformers/blob/master/src/transformers/training_args.py#L33) is referenced here. 

In [37]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./test_RoBERTa2",
    overwrite_output_dir=True,  #"Use this to continue training if output_dir points to a checkpoint directory."
    
    
    do_train=True, #Whether to run training.
    do_eval=True, #Whether to run eval on the dev set.
#     do_predict=True, # Whether to run predictions on the test set.
    
    num_train_epochs=20, # Total number of training epochs to perform.
    
    
    per_device_train_batch_size=8, # Batch size per GPU/TPU core/CPU for training.
    per_device_eval_batch_size=8, # Batch size per GPU/TPU core/CPU for evaluation.
    
    learning_rate=5e-5,  #The initial learning rate for Adam.
    adam_epsilon=1e-8, #Epsilon for Adam optimizer.
    
    save_steps=10_000,  #Save checkpoint every X updates steps.
    save_total_limit=2, #"Limit the total amount of checkpoints. Deletes the older checkpoints in the output_dir. Default is unlimited checkpoints
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
%%time
trainer.train()

In [ ]:
trainer.save_model("./EsperBERTo")

In [ ]:
encoded = pretrain_tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces")
encoded